# Gibbs Sampler

In [1]:
%reset -f
%matplotlib inline

from enum import Enum
import matplotlib as mpl
import numpy as np
from numpy import array as a
import matplotlib.pyplot as plt
import numpy.random as rng
from scipy.special import expit as sigmoid
np.set_printoptions(precision = 2, suppress = True)
import time
from functools import reduce
import operator



## Base parameter values for gibbs


In [2]:
wbase = 0.4
wlo =  0.1
whigh = 0.8

## Network/Nodes
The section below is how nodes in the network are represented, in this case 
the network supports only the logcal operators.

In [3]:
""" The four types of allowable nodes, we are modeling a graph which supports the logical operators"""
class NodeType(Enum):
    MARGINAL = 1, # A node with no parents
    NOT = 2,
    OR = 3,
    AND = 4

""" Represents a node in the PGM """
class Node:
    
    def __init__(self, state, parents, func):
        self.state = state
        self.parents = parents
        
        # Different function based on node type
        if func == NodeType.MARGINAL:
            self.func = marginalProbability
        elif func == NodeType.NOT:
            self.func = notProbability
        elif func == NodeType.OR:
            self.func = orProbability
        elif func == NodeType.AND:
            self.func = andProbability
        else:
            print("Not a valid type")
        
    def probability(self):
        return self.func(self.parents)
        

''' The various node type proababilities based on the parents'''
def marginalProbability(p):
    return wbase

def notProbability(p):
    if p[0].state:
        return wlo
    else:
        return whigh
    
def orProbability(p):
    if p[0].state or p[1].state:
        return whigh
    else:
        return wlo
    
def andProbability(p):
    if p[0].state and p[1].state and p[2].state:
        return whigh
    else:
        return wlo


## Network
Construct a sample network, in this case there are 10 nodes in the network. 
Nodes should be referenced by a unique name (in this case "Xi", where i is unique across all nodes).

In [4]:
PGM = {}
PGM["X1"] = Node(0, [], NodeType.MARGINAL)
PGM["X2"] = Node(0, [], NodeType.MARGINAL)
PGM["X3"] = Node(0, [], NodeType.MARGINAL)
PGM["X4"] = Node(0, [PGM["X1"]], NodeType.NOT)
PGM["X5"] = Node(0, [], NodeType.MARGINAL)
PGM["X6"] = Node(0, [PGM["X1"], PGM["X2"]], NodeType.OR)
PGM["X7"] = Node(0, [], NodeType.MARGINAL)
PGM["X8"] = Node(0, [PGM["X3"], PGM["X4"], PGM["X5"]], NodeType.AND)
PGM["X9"] = Node(0, [PGM["X5"], PGM["X6"], PGM["X7"]], NodeType.AND)
PGM["X10"] = Node(0, [PGM["X8"], PGM["X9"]], NodeType.OR)

## Gibbs Sampler

The main code for gibbs sampling is below, the method takes a goal (i.e. a state of a particular node) and an optional set of observed nodes. 

In [5]:
def gibbs(goal, observed):
    
    (goalNode, goalState) = goal
    
    count = 0
    
    iterations = 1000 * 2 ** len(PGM)
    
    # All the nodes in the PGM
    keys = list(PGM)
    
    # Mark the nodes we have observed
    for (key, val) in observed:
        PGM[key].state = val
        # Now we do not want to change these node states, so remove from the keyset
        keys.remove(key)
        
    # The number of remaining keys
    size = len(keys)

    for _ in range(iterations):

        # Choose random node to reset
        index = keys[rng.randint(size)]
        node = PGM[index]

        #P(X with the random node = 1)
        totalProbOne = totalProb(index, 1, PGM)
        #P(X with the random node = 0)
        totalProbZero = totalProb(index, 0, PGM)

        #P(random node = 1 | X\random node)
        oneProb = totalProbOne / (totalProbOne + totalProbZero)
        
        # If the random number is below the probability of a one
        # set it to one, Else set it to 0
        node.state = 1 * (rng.random() <= oneProb)

        if PGM[goalNode].state == goalState:
            count +=1
            
    return count / iterations


# Calculate the probability of P(Xi=1 | X\i)
def totalProb(i, state, xs):
    originalState = xs[i].state
    xs[i].state = state
    probs = [ x.probability() if x.state else 1 - x.probability() for x in xs.values()]
    product = reduce(operator.mul, probs, 1)
    xs[i].state = originalState
    return product

## Examples

Some testing examples to see how Gibbs estiamtes the probabilities

In [1]:
prob = gibbs(("X2" ,1), [("X4", 1), ("X6", 1)])
print(prob)

prob = gibbs(("X2" ,1), [("X6", 1)])
print(prob)

NameError: name 'gibbs' is not defined